In [ ]:
# This notebook will leverage nursing data from CMS to draw on several insights

# Nursing Burnout and its impact on Nurse Lifestyles
# https://ldi.upenn.edu/our-work/research-updates/how-inadequate-hospital-staffing-continues-to-burn-out-nurses-and-threaten-patients/#:~:text=Before%20the%20pandemic%2C%2064.9%25%20of,to%2058.9%25%20during%20the%20pandemic.

# Data Methodology
# https://data.cms.gov/resources/payroll-based-journal-methodology-0

# Data Dictionaries / Glossaries / indexes
# https://data.cms.gov/resources/payroll-based-journal-daily-non-nurse-staffing-data-dictionary
# https://data.cms.gov/resources/payroll-based-journal-daily-nurse-staffing-data-dictionary
# https://www.kaggle.com/datasets/miadul/overstimulation-behavior-and-lifestyle-dataset
# https://www.kaggle.com/datasets/manuelcamachor/level-of-anxiety-in-nursing-during-covid-pandemic

# Datasets
# https://data.cms.gov/quality-of-care/payroll-based-journal-daily-non-nurse-staffing/data
# https://data.cms.gov/quality-of-care/payroll-based-journal-daily-nurse-staffing/data
# https://www.kaggle.com/datasets/miadul/overstimulation-behavior-and-lifestyle-dataset
# https://www.kaggle.com/datasets/manuelcamachor/level-of-anxiety-in-nursing-during-covid-pandemic

# Hypothesis
# Using a combination of real and synthetic data, we aspire to show that nurses who work in under-staffed medical settings, experience negative lifestyle effects by being overstimulated / having a low nurse to patient index (something like this - explore the data)

In [ ]:
## Task:
#1. Data Familiarity & Exploration  
# • Load and explore the dataset (can be any public dataset or one provided by you – e.g., UCI, Kaggle). 
# • Identify key trends and what the data might say about the user. 

#2. Build a Simple Model  
# • Choose a modeling goal: e.g., user segmentation, outcome prediction, or recommendation. 
# • Select and justify a model (logistic regression, decision trees, simple neural net, etc.). 
# • Train and validate (simple train/test split is fine). 

#3. Evaluate & Explain  
# • Use one or two performance metrics (e.g., accuracy, F1, ROC AUC). 
# • Explain the why behind your modeling choices. 
# • Summarize key insights (via comments, markdown, or a short PDF summary). 

#4. Final Thoughts  
# • Briefly discuss how you’d productionize this in a startup setting.

In [1]:
from distributed.dashboard.components.shared import Processing
# Setup
!pip install polars


   ---------------------------------------- 0.0/35.6 MB ? eta -:--:--
   ------- -------------------------------- 7.1/35.6 MB 48.3 MB/s eta 0:00:01
   --------------------- ------------------ 19.1/35.6 MB 52.6 MB/s eta 0:00:01
   ----------------------------------- ---- 31.5/35.6 MB 54.0 MB/s eta 0:00:01
   ---------------------------------------- 35.6/35.6 MB 49.2 MB/s eta 0:00:00


DEPRECATION: Loading egg at c:\users\tmele\anaconda3\lib\site-packages\mask_rcnn-2.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


## Get Data


In [39]:
# This Section is just for me to download the source data, the included datasets replace these files and are much smaller. There's no need to use this section other than to review how the api was accessed to download the data.

# Getting the data via api
import polars as pl
import requests
import os
import time

# define datasets for api acquisition
non_nursing_data = [
    {
        "type": "Non-Nursing Staffing",
        "quarter": "Q3",
        "year": "2024",
        "id": "293788f3-20a9-4ebd-ab4a-9989325ce3e7"
    },
    {
        "type": "Non-Nursing Staffing",
        "quarter": "Q2",
        "year": "2024",
        "id": "c06ad548-5cb2-4f29-8075-66efc2d195b6"
    },
    {
        "type": "Non-Nursing Staffing",
        "quarter": "Q1",
        "year": "2024",
        "id": "86603f7e-6ed8-4005-a5d3-b13d2ebcf3d4"
    }
]
non_nursing_data = [non_nursing_data[2]]
nursing_data = [
    {
        "type": "Nursing Staffing",
        "quarter": "Q3",
        "year": "2024",
        "id": "989fbc78-1655-487d-9f24-d68e9a0ab3af"
    },
    {
        "type": "Nursing Staffing",
        "quarter": "Q2",
        "year": "2024",
        "id": "dcc467d8-5792-4e5d-95be-04bf9fc930a1"
    },
    {
        "type": "Nursing Staffing",
        "quarter": "Q1",
        "year": "2024",
        "id": "5b84bdf2-b246-4b3c-be1b-cf7c2bcb3391"
    }
]
nursing_data = [nursing_data[2]]

data_sets = nursing_data + non_nursing_data

# optional
data_frames = []

# directory of where this notebook will be run
base_dir = os.getcwd()+"\\data\\"

# avoid extra api calls
for item in data_sets:
    file = [str(val) for val in item.values()]
    file = "-".join(file)+".tsv"
    this_file = base_dir+file
    
    if os.path.exists(this_file):
        pass # df = pl.read_csv(this_file, separator='\t')
        # data_frames.append(df)
    else:
        _id = item['id']
        
        # get the base url
        api_url = "https://data.cms.gov/data-api/v1/dataset/"+_id+"/data"
        
        # places to store data
        _data = []
        
        # get number of rows
        response = requests.get(api_url+"/stats")
        rows = response.json()
        total_rows = rows.get("total_rows")
        i = 0 
        
        # api docs stated they can provide 5k per call but only 1k is in the response
        print(f"obtaining {total_rows} rows 1000 at a time")
        while i < total_rows: 
            size = 1000
            offset_url = f"{api_url}?/size={size}&offset={i}"
            offset = i
            offset_response = requests.get(offset_url)
            print(f"Made request for {size} results at offset {i}")
            items = offset_response.json()
            for item in items:
                _data.append(item)
            i = i+size
            print(["Size is now:",len(_data)])
            time.sleep(0.5)     
            
        df = pl.DataFrame(_data)
        df.write_csv(this_file,include_header=True,separator="\t",line_terminator="\n",quote_char='"',null_value="NA")
        time.sleep(120)


obtaining 1330966 rows 1000 at a time
Made request for 1000 results at offset 0
['Size is now:', 1000]
Made request for 1000 results at offset 1000
['Size is now:', 2000]
Made request for 1000 results at offset 2000
['Size is now:', 3000]
Made request for 1000 results at offset 3000
['Size is now:', 4000]
Made request for 1000 results at offset 4000
['Size is now:', 5000]
Made request for 1000 results at offset 5000
['Size is now:', 6000]
Made request for 1000 results at offset 6000
['Size is now:', 7000]
Made request for 1000 results at offset 7000
['Size is now:', 8000]
Made request for 1000 results at offset 8000
['Size is now:', 9000]
Made request for 1000 results at offset 9000
['Size is now:', 10000]
Made request for 1000 results at offset 10000
['Size is now:', 11000]
Made request for 1000 results at offset 11000
['Size is now:', 12000]
Made request for 1000 results at offset 12000
['Size is now:', 13000]
Made request for 1000 results at offset 13000
['Size is now:', 14000]
Made

In [ ]:
# this exists for validation as much as a way to label things and for validation. It is useful.

data_definitions = {
    "PROVNUM": "Medicare provider number",
    "PROVNAME": "Provider name",
    "CITY": "Provider City",
    "STATE": "Postal abbreviation for State",
    "COUNTY_NAME": "Name of Provider County, unique within state",
    "COUNTY_FIPS": "FIPS Code for Provider County, unique within state",
    "CY_Qtr": "Calendar Quarter (yyyyQq, e.g. 2018Q4)",
    "WorkDate": "Day for Reported Hours (yyyymmdd)",
    "MDScensus": "Resident Census from MDS",
    "Hrs_RNDON": "Total Hours for RN Director of Nursing", # no EMP?
    "Hrs_RNDON_emp": "Employee Hours for RN Director of Nursing", # this was missing and confirmed below
    "Hrs_RNDON_ctr": "Contract Hours for RN Director of Nursing",
    "Hrs_RNadmin": "Hours for RN with administrative duties",
    "Hrs_RNadmin_emp": "Employee Hours for RN with administrative duties",
    "Hrs_RNadmin_ctr": "Contract Hours for RN with administrative duties",
    "Hrs_RN": "Total Hours for RN",
    "Hrs_RN_emp": "Employee Hours for RN",
    "Hrs_RN_ctr": "Contract Hours for RN",
    "Hrs_LPNadmin": "Total Hours for LPN w/ admin duties",
    "Hrs_LPNadmin_emp": "Employee Hours for LPN w/ admin duties",
    "Hrs_LPNadmin_ctr": "Contract Hours for LPN w/ admin duties",
    "Hrs_LPN": "Total Hours for LPN",
    "Hrs_LPN_emp": "Employee Hours for LPN",
    "Hrs_LPN_ctr": "Contract Hours for LPN",
    "Hrs_CNA": "Total Hours for CNA",
    "Hrs_CNA_emp": "Employee Hours for CNA",
    "Hrs_CNA_ctr": "Contract Hours for CNA",
    "Hrs_NAtrn": "Total Hours for Nurse aide in training",
    "Hrs_NAtrn_emp": "Employee Hours for Nurse aide in training",
    "Hrs_NAtrn_ctr": "Contract Hours for Nurse aide in training",
    "Hrs_MedAide": "Total Hours for Med Aide/Technician",
    "Hrs_MedAide_emp": "Employee Hours for Med Aide/Technician",
    "Hrs_MedAide_ctr": "Contract Hours for Med Aide/Technician",
    #
    "Hrs_Admin": "Total Hours for Administrator",
    "Hrs_Admin_emp": "Employee Hours for Administrator",
    "Hrs_Admin_ctr": "Contract Hours for Administrator",
    "Hrs_Admin fn": "Footnote for Administrator Hours worked: 1 = Provider submitted invalid administrator hours",
    "Hrs_MedDir": "Total Hours for Medical Director",
    "Hrs_MedDir_emp": "Employee Hours for Medical Director",
    "Hrs_MedDir_ctr": "Contract Hours for Medical Director",
    "Hrs_OthMD": "Total Hours for Other Physician",
    "Hrs_OthMD_emp": "Employee Hours for Other Physician",
    "Hrs_OthMD_ctr": "Contract Hours for Other Physician",
    "Hrs_PA": "Total Hours for Physician Assistant",
    "Hrs_PA_emp": "Employee Hours for Physician Assistant",
    "Hrs_PA_ctr": "Contract Hours for Physician Assistant",
    "Hrs_NP": "Total Hours for Nurse Practitioner",
    "Hrs_NP_emp": "Employee Hours for Nurse Practitioner",
    "Hrs_NP_ctr": "Contract Hours for Nurse Practitioner",
    "Hrs_ClinNrsSpec": "Total Hours for Clinical Nurse Specialist",
    "Hrs_ClinNrsSpec_emp": "Employee Hours for Clinical Nurse Specialist",
    "Hrs_ClinNrsSpec_ctr": "Contract Hours for Clinical Nurse Specialist",
    "Hrs_Pharmacist": "Total Hours for Pharmacist",
    "Hrs_Pharmacist_emp": "Employee Hours for Pharmacist",
    "Hrs_Pharmacist_ctr": "Contract Hours for Pharmacist",
    "Hrs_Dietician": "Total Hours for Dietician",
    "Hrs_Dietician_emp": "Employee Hours for Dietician",
    "Hrs_Dietician_ctr": "Contract Hours for Dietician",
    "Hrs_FeedAsst": "Total Hours for Feeding Assistant",
    "Hrs_FeedAsst_emp": "Employee Hours for Feeding Assistant",
    "Hrs_FeedAsst_ctr": "Contract Hours for Feeding Assistant",
    "Hrs_OT": "Total Hours for Occupational Therapist",
    "Hrs_OT_emp": "Employee Hours for Occupational Therapist",
    "Hrs_OT_ctr": "Contract Hours for Occupational Therapist",
    "Hrs_OTasst": "Total Hours for Occupational Therapy Assistant",
    "Hrs_OTasst_emp": "Employee Hours for Occupational Therapy Assistant",
    "Hrs_OTasst_ctr": "Contract Hours for Occupational Therapy Assistant",
    "Hrs_OTaide": "Total Hours for Occupational Therapy Aide",
    "Hrs_OTaide_emp": "Employee Hours for Occupational Therapy Aide",
    "Hrs_OTaide_ctr": "Contract Hours for Occupational Therapy Aide",
    "Hrs_PT": "Total Hours for Physical Therapist",
    "Hrs_PT_emp": "Employee Hours for Physical Therapist",
    "Hrs_PT_ctr": "Contract Hours for Physical Therapist",
    "Hrs_PTasst": "Total Hours for Physical Therapy Assistant",
    "Hrs_PTasst_emp": "Employee Hours for Physical Therapy Assistant",
    "Hrs_PTasst_ctr": "Contract Hours for Physical Therapy Assistant",
    "Hrs_PTaide": "Total Hours for Physical Therapy Aide",
    "Hrs_PTaide_emp": "Employee Hours for Physical Therapy Aide",
    "Hrs_PTaide_ctr": "Contract Hours for Physical Therapy Aide",
    "Hrs_RespTher": "Total Hours for Respiratory Therapist",
    "Hrs_RespTher_emp": "Employee Hours for Respiratory Therapist",
    "Hrs_RespTher_ctr": "Contract Hours for Respiratory Therapist",
    "Hrs_RespTech": "Total Hours for Respiratory Therapy Technician",
    "Hrs_RespTech_emp": "Employee Hours for Respiratory Therapy Technician",
    "Hrs_RespTech_ctr": "Contract Hours for Respiratory Therapy Technician",
    "Hrs_SpcLangPath": "Total Hours for Speech/Language Pathologist",
    "Hrs_SpcLangPath_emp": "Employee Hours for Speech/Language Pathologist",
    "Hrs_SpcLangPath_ctr": "Contract Hours for Speech/Language Pathologist",
    "Hrs_TherRecSpec": "Total Hours for Therapeutic Recreation Specialist",
    "Hrs_TherRecSpec_emp": "Employee Hours for Therapeutic Recreation Specialist",
    "Hrs_TherRecSpec_ctr": "Contract Hours for Therapeutic Recreation Specialist",
    "Hrs_QualActvProf": "Total Hours for Qualified Activities Professional",
    "Hrs_QualActvProf_emp": "Employee Hours for Qualified Activities Professional",
    "Hrs_QualActvProf_ctr": "Contract Hours for Qualified Activities Professional",
    "Hrs_OthActv": "Total Hours for Other Activities Staff",
    "Hrs_OthActv_emp": "Employee Hours for Other Activities Staff",
    "Hrs_OthActv_ctr": "Contract Hours for Other Activities Staff",
    "Hrs_QualSocWrk": "Total Hours for Qualified Social Worker",
    "Hrs_QualSocWrk_emp": "Employee Hours for Qualified Social Worker",
    "Hrs_QualSocWrk_ctr": "Contract Hours for Qualified Social Worker",
    "Hrs_OthSocWrk": "Total Hours for Other Social Worker",
    "Hrs_OthSocWrk_emp": "Employee Hours for Other Social Worker",
    "Hrs_OthSocWrk_ctr": "Contract Hours for Other Social Worker",
    "Hrs_MHSvc": "Total Hours for Mental Health Service Worker",
    "Hrs_MHSvc_emp": "Employee Hours for Mental Health Service Worker",
    "Hrs_MHSvc_ctr": "Contract Hours for Mental Health Service Worker"
}   

In [117]:
# Transform Data into smaller usable components for analysis
import polars as pl


# directory of where this notebook will be run
base_dir = os.getcwd()+"\\data\\"

nurses = pl.read_csv(base_dir+"Nursing Staffing-Q1-2024-5b84bdf2-b246-4b3c-be1b-cf7c2bcb3391.tsv", separator="\t", infer_schema_length=10000,ignore_errors=True)
non_nurses = pl.read_csv(base_dir+"Non-Nursing Staffing-Q2-2024-86603f7e-6ed8-4005-a5d3-b13d2ebcf3d4.tsv", separator="\t", infer_schema_length=10000,ignore_errors=True)

################################################## Inspect Data ##################################################
################################################## Inspect Data ##################################################

# let's work with real places not using tax credits to possibly steal $$$???
nurses = nurses.filter(pl.col("PROVNUM").is_not_null())
non_nurses = non_nurses.filter(pl.col("PROVNUM").is_not_null())

# the max provnum is for prov 335232, the data dict does not explicitly state the emp and ctr fields add to total
# this could be assumed but, check first to ensure understanding is clear. these three expression yield this is true
# and our thinking was correct. Every role in the data follows this format so that clears up basically all the fields
_nurses = nurses.group_by("PROVNUM").agg(pl.sum('Hrs_RN').alias('Registered Nurses')).sort('Registered Nurses', descending=True)
print("Total Hours:",_nurses[0]["Registered Nurses"][0])
__nurses = nurses.group_by("PROVNUM").agg(pl.sum('Hrs_RN_emp').alias('Registered Nurses emp')).filter(pl.col('PROVNUM') == 335232)
print("Employee Hours:",__nurses[0]["Registered Nurses emp"][0])
___nurses = nurses.group_by("PROVNUM").agg(pl.sum('Hrs_RN_ctr').alias('Registered Nurses ctr')).filter(pl.col('PROVNUM') == 335232)
print("Contract Hours:",___nurses[0]["Registered Nurses ctr"][0])

# is the difference really small because of rounding only
if _nurses[0]["Registered Nurses"][0] - __nurses[0]["Registered Nurses emp"][0] - ___nurses[0]["Registered Nurses ctr"][0] < 1:
    print(True)
else:
    print(False)

# this field from non nurses is different   
#    "Hrs_Admin fn": "Footnote for Administrator Hours worked: 1 = Provider submitted invalid administrator hours",
# what do they mean by this, inspect
_non_nurses = non_nurses.filter(pl.col("Hrs_Admin_fn") != "1")['Hrs_Admin_fn']
no_mistake = len(_non_nurses)
_non_nurses = non_nurses.filter(pl.col("Hrs_Admin_fn") == "1")['Hrs_Admin_fn']
mistake = len(_non_nurses)

print("the % of mistakes in the Admin field is:", mistake/no_mistake*100) # <1% is basically irrelevant, but could have been

# I don't see a "Hrs_RNDON_emp" in my data dictionary above - lets check if I missed copying it or if it is not there
if "Hrs_RNDON_emp" in nurses.columns:
    print("Hrs_rn_emp is in nurses")
# it is in the columns, I made a copy/paste error in making my data dict - I will add it back and note it



################################################## Data Insights ##################################################
################################################## Data Insights ##################################################










Total Hours: 45625.189999999995
Employee Hours: 35137.75000000001
Contract Hours: 10487.44
True
the % of mistakes in the Admin field is: 0.6361856823266219
Hrs_rn_emp is in nurses
